In [1]:
import os
import pandas as pd
import numpy as np
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 194513


,eng,fra,cc
105123,We have a party this evening.,Nous tenons une fête ce soir.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
172602,I couldn't put up with her arrogant behavior.,Je ne pouvais supporter son attitude arrogante.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
179018,You're not the only one who doesn't believe Tom.,Vous n'êtes pas la seule à ne pas croire Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
157201,It is easy for me to solve the problem.,Il m'est facile de résoudre le problème.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
158251,Tom should have followed Mary's advice.,Tom aurait dû suivre les conseils de Marie.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
29588,I jog twice a week.,Je cours deux fois par semaine.
10912,I'm illiterate.,Je suis analphabète.
12751,You're a prude.,Vous êtes une puritaine.
12793,You're hideous.,Vous êtes hideuse.
6998,He tricked me.,Il m'a eue.


**Preprocessing**

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

In [5]:
def preprocess_sentence_decoder(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    sentence = sentence.split(" ")
    
    return sentence

In [6]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence_decoder(x))

In [7]:
lines.eng.sample(5)

9002            [was, tom, happy, ?]
30706     [it, s, a, distraction, .]
28419    [don, t, be, so, stingy, .]
2542                [try, harder, .]
27463     [you, are, in, my, way, .]
Name: eng, dtype: object

In [8]:
lines.fra.sample(5)

30970    [<start>, regarde, ce, qui, est, arriv, ., <end>]
7947     [<start>, c, est, ici, que, a, fait, mal, ., <...
30045    [<start>, je, voulais, des, chaussures, rouges...
4384                   [<start>, es, tu, chauve, ?, <end>]
20166            [<start>, quelqu, un, a, rigol, ., <end>]
Name: fra, dtype: object

**Vocabulary**

In [9]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)

In [10]:
print(eng_tokenizer.word_index)

{'.': 1, 'i': 2, 'you': 3, '?': 4, 'tom': 5, 'it': 6, 'is': 7, 's': 8, 'a': 9, 're': 10, 'we': 11, 'm': 12, 't': 13, 'the': 14, 'that': 15, 'he': 16, 'was': 17, 'me': 18, 'this': 19, 'they': 20, 'to': 21, 'are': 22, '!': 23, 'do': 24, 'can': 25, 'my': 26, 'your': 27, 'have': 28, 'go': 29, 'not': 30, 'don': 31, 'll': 32, 'be': 33, 'she': 34, 'no': 35, 'what': 36, 'did': 37, 'all': 38, 'here': 39, 'let': 40, 'like': 41, 'in': 42, 'get': 43, 'up': 44, 'very': 45, 'how': 46, 'on': 47, 'need': 48, 'one': 49, 'want': 50, 'him': 51, 'out': 52, 'love': 53, 'who': 54, 'come': 55, 'there': 56, 'just': 57, 'please': 58, 'now': 59, 'us': 60, 'know': 61, 'stop': 62, 'so': 63, 'of': 64, 'got': 65, 'too': 66, 'help': 67, 'look': 68, 'take': 69, 'were': 70, 'good': 71, 'for': 72, 'has': 73, 've': 74, 'see': 75, 'at': 76, 'will': 77, 'keep': 78, 'am': 79, 'had': 80, 'back': 81, 'give': 82, 'feel': 83, 'try': 84, 'stay': 85, 'saw': 86, 'must': 87, 'happy': 88, 'home': 89, 'where': 90, 'alone': 91, 'didn

In [11]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:3]

[[29, 1], [29, 1], [29, 1]]

In [12]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)

In [13]:
target_text[:3]

[[1, 75, 8, 2], [1, 378, 3, 2], [1, 718, 8, 2]]

In [14]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7460


In [15]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [16]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7460
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


**for teacher forcing**

In [17]:
sos_token = '<start>'
eos_token = '<end>'

encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

**padding**

In [18]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


**train validation split**

In [19]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [20]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


**Train encoder**

In [21]:
encoder_inputs = Input(shape=(None,))
# encoder embedding
enc_emb = Embedding(eng_vocab_size, 256, input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

**Train decoder**

In [22]:
decoder_inputs = Input(shape=(None,))
# decoder embedding
dec_emb = Embedding(fra_vocab_size, 256)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1195776     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1909760     input_2[0][0]                    
______________________________________________________________________________________________

In [24]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [25]:
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
938/938 [==============================] - 27s 19ms/step - loss: 1.4708 - acc: 0.7713 - val_loss: 1.1928 - val_acc: 0.8130
Epoch 2/50
938/938 [==============================] - 16s 17ms/step - loss: 1.1070 - acc: 0.8243 - val_loss: 1.0362 - val_acc: 0.8350
Epoch 3/50
938/938 [==============================] - 15s 17ms/step - loss: 0.9790 - acc: 0.8421 - val_loss: 0.9553 - val_acc: 0.8478
Epoch 4/50
938/938 [==============================] - 15s 17ms/step - loss: 0.8950 - acc: 0.8533 - val_loss: 0.8972 - val_acc: 0.8547
Epoch 5/50
938/938 [==============================] - 15s 16ms/step - loss: 0.8320 - acc: 0.8624 - val_loss: 0.8633 - val_acc: 0.8595
Epoch 6/50
938/938 [==============================] - 15s 17ms/step - loss: 0.7830 - acc: 0.8701 - val_loss: 0.8272 - val_acc: 0.8649
Epoch 7/50
938/938 [==============================] - 16s 17ms/step - loss: 0.7469 - acc: 0.8770 - val_loss: 0.8113 - val_acc: 0.8697
Epoch 8/50
938/938 [==============================] - 15s 16ms

**Test encoder**

In [26]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1195776   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,721,088
Trainable params: 1,721,088
Non-trainable params: 0
_________________________________________________________________


**Test decoder**

In [28]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [29]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    1909760     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [30]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [31]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [32]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [33]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [34]:
for seq_index in [1,100,301,777,2222]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: he was all wrong . 
정답 문장: il avait tout faux . 
번역기가 번역한 문장:  tout te restez j 
-----------------------------------
입력 문장: you re very smart . 
정답 문장: tu es tr s intelligent . 
번역기가 번역한 문장:  vous tes es cela 
-----------------------------------
입력 문장: you re good . 
정답 문장: vous tes bonne . 
번역기가 번역한 문장:  vous bon bien . 
-----------------------------------
입력 문장: he heard a noise . 
정답 문장: il a entendu un bruit . 
번역기가 번역한 문장:  il marcher des . 
-----------------------------------
입력 문장: have some ham . 
정답 문장: prenez du jambon . 
번역기가 번역한 문장:  prends la . . . 


**회고**

sequence-to-sequence 모델을 통해서 뉴럴 기계번역을 조금 더 이해할 수 있는 흥미로운 시간이었다.